In [1]:
import pandas as pd
A=pd.read_csv("C:/Users/Suja4/OneDrive/Documents/Data Science/Dataset/cars93.csv")

from warnings import filterwarnings
filterwarnings("ignore")

In [2]:
from Wd8pm import replacer
replacer(A)

In [3]:
Y = A[["Origin"]]
X = A.drop(labels=["id","Make","Model","Origin"],axis=1)
from Wd8pm import prep_NN
Xnew,Ynew = prep_NN(X,Y)

In [4]:
from keras.models import Sequential
from keras.layers import Dense,Dropout
import keras_tuner as kt
from keras.callbacks import EarlyStopping

In [5]:
def model_builder(hp):
    nn = Sequential()
    nn.add(Dense(200,input_dim=(len(Xnew.columns))))
    nn.add(Dropout(0.1))
    x = hp.Int("units",min_value=32,max_value=512,step=32)
    nn.add(Dense(units=x))
    nn.add(Dense(1,activation="sigmoid"))
    nn.compile(loss="binary_crossentropy",metrics="accuracy")
    return nn

In [6]:
tuner = kt.Hyperband(model_builder,
            objective="val_accuracy",
            max_epochs=5,
            directory="Desktop/KT1",
            project_name="KT_ex1")

INFO:tensorflow:Reloading Oracle from existing project Desktop/KT1\KT_ex1\oracle.json
INFO:tensorflow:Reloading Tuner from Desktop/KT1\KT_ex1\tuner0.json


In [7]:
es = EarlyStopping(monitor="accuracy",patience=10)
tuner.search(Xnew,Ynew,validation_split=0.2,epochs=10,callbacks=[es])

INFO:tensorflow:Oracle triggered exit


In [8]:
q = tuner.get_best_hyperparameters(num_trials=11)[0]
q.get("units")

448